<a href="https://colab.research.google.com/github/amalvarezme/AnaliticaDatos/blob/master/FaceRecognition/FaceTagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python face_recognition -q

In [ ]:
!git clone https://github.com/amalvarezme/FaceRecognition.git

In [ ]:
import cv2
import face_recognition
import matplotlib.pyplot as plt
import os
import numpy as np

# Load the labeled face image and encode it
labeled_image_path = 'FaceRecognition/gcpds.jpeg'  # Replace with the path to your labeled face image
labeled_image = face_recognition.load_image_file(labeled_image_path)
labeled_image = cv2.resize(labeled_image, (2048,1152))

plt.imshow(labeled_image)
plt.show()

In [ ]:
face_locations = face_recognition.face_locations(labeled_image, number_of_times_to_upsample=1)
print(f'face detected: {len(face_locations)}')

In [ ]:
# @title
def loc_faces(image_,face_locations,linew_=2,font_s=2,save_=True,output_dir=None,h_=80,w_=80):
  # Loop over each face found in the image
  i = 0
  Nf = len(face_locations)
  faces = np.zeros((Nf,h_,w_,image_.shape[-1]))

  if save_:
    for (top, right, bottom, left) in face_locations:
        print(f'face {i+1}/{Nf}')

        # Extract the face from the frame
        face_image = cv2.resize(cv2.cvtColor(image_[top:bottom, left:right], cv2.COLOR_RGB2BGR),(h_, w_))

        # Save the face image as a file
        face_filename = os.path.join(output_dir, f"face_{i+1}.jpg")
        cv2.imwrite(face_filename, face_image)
        faces[i] = face_image
        i+=1
  i=0
  for (top, right, bottom, left) in face_locations:
    print(f'face {i+1}/{Nf}')
    # Draw a rectangle around the matching face
    cv2.rectangle(image_, (left, top), (right, bottom), (0, 255, 0), linew_)
    #cv2.putText(image_, 'f'+str(i+1), (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, font_s, (0, 255, 0), linew_)
    i+=1

  return image_,faces

In [ ]:
# Create a directory to save detected faces
%matplotlib inline
output_dir = "gcpds_detected_faces"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


image_ = labeled_image.copy()

image_,faces = loc_faces(image_,face_locations,
                         save_= True,output_dir=output_dir)
plt.imshow(image_)
plt.show()

In [ ]:
!pip install ipympl -q
from google.colab import output
output.enable_custom_widget_manager()


In [ ]:
#@title
#bounding box faces not detected

import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Global variable to store the selected points
ref_points = []
ref_points_ = []

# Mouse event handler function
def onclick(event):
    global ref_points, ref_points_

    # If you click inside the axes
    if event.xdata and event.ydata:
        # Store the click position as (x, y)
        ref_points_.append((int(event.xdata), int(event.ydata)))
        print(ref_points_)

        # Once two points are clicked, draw the rectangle
        if len(ref_points_) == 2:
           ref_points.append(ref_points_)
           ref_points_ = []
           draw_rectangle(ref_points[-1])


# Function to draw the bounding box after two clicks
def draw_rectangle(points):


    (x1, y1), (x2, y2) = points

    width = x2 - x1
    height = y2 - y1

    # Draw the rectangle on the same figure without resetting
    rect = Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')


    # Add the rectangle to the plot and refresh the figure
    ax.add_patch(rect)
    fig.canvas.draw()



In [ ]:
%matplotlib inline
plt.imshow(image_)
plt.show()

%matplotlib ipympl

# Convert the image from BGR to RGB for displaying with matplotlib
image_rgb = labeled_image.copy()# cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
fig, ax = plt.subplots()
ax.imshow(image_rgb)

# Connect the click event to the figure
cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()


In [ ]:
ref_points

In [ ]:
face_locations_ = face_locations


In [ ]:
any(ref_points)

In [ ]:
if any(ref_points):
  for ref_add in ref_points:
    (left, top), (right, bottom) = ref_add
    print(top,bottom,left,right)
    face_locations_.append((top,right,bottom,left))

In [ ]:
%matplotlib inline
image2_ = labeled_image.copy()
image2_,faces = loc_faces(image2_,face_locations_,
                         save_= True,output_dir=output_dir)
plt.imshow(image2_)
plt.show()

In [ ]:
face_encoding_ = np.array(face_recognition.face_encodings(labeled_image, known_face_locations=face_locations_))  # Get face encoding for the labeled face


In [ ]:
plt.pcolormesh(face_encoding_)
plt.colorbar()
plt.show()

In [ ]:
!pip install umap-learn -q #librería umap

In [ ]:
import umap
red = umap.UMAP(n_components=2,n_neighbors=3, min_dist=0.2)
Z = red.fit_transform(face_encoding_)

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox #plot mnist 2D
def plot_mnist_2d(Z,y,images,img_w=28,img_h=28,zoom=0.5,cmap='jet'):
    fig, ax = plt.subplots(figsize=(8,5))
    for i in range(Z.shape[0]):
        #print('img',i+1,'/',Z.shape[0])
        image =  cv2.resize(images[i],(img_w,img_h))
        im = OffsetImage(image[:,:,[2,1,0]], zoom=zoom,cmap=cmap)
        ab = AnnotationBbox(im, (Z[i,0], Z[i,1]), xycoords='data', frameon=False)
        ax.add_artist(ab)
        ax.update_datalim([(Z[i,0], Z[i,1])])
        ax.autoscale()

In [ ]:
plot_mnist_2d(Z,np.ones((Z.shape[0])),faces/255.)

In [ ]:
import openpyxl
from openpyxl.drawing.image import Image as ExcelImage
from PIL import Image
import numpy as np
import io


def excel_faces(dir_,faces):
  # Create a new workbook and select the active worksheet
  wb = openpyxl.Workbook()
  ws = wb.active

  # Loop over the images in the array and insert them into the Excel file
  for row in range(faces.shape[0]):
      # Convert the NumPy image to a PIL image
      img_array = np.uint8(faces[row])
      img_pil = Image.fromarray(img_array[:,:,[2,1,0]])

      # Save the PIL image into a bytes buffer
      img_buffer = io.BytesIO()
      img_pil.save(img_buffer, format='PNG')
      img_buffer.seek(0)

      # Convert the bytes buffer into an image that openpyxl can use
      img_excel = ExcelImage(img_buffer)

      # Resize the image to 80x80
      img_excel.width, img_excel.height = 80, 80

      # Insert the image into the worksheet, in column 'A'
      cell = f'A{row+2}'
      ws.add_image(img_excel, cell)

      # Insert the face ID in column 'B'
      cell_face_id = f'B{row+2}'
      ws[cell_face_id] = row+2

      # Insert the name in column 'C'
      cell_name = f'C{row+2}'
      plt.imshow(img_array[:,:,[2,1,0]])
      plt.show()

      name_ = input('name: ')
      ws[cell_name] = name_


      # Set the row height to 80




  ws.column_dimensions['A'].width = 12.0  # Adjusted to match 80px roughly for Excel
  # Set the column width for 'B' and 'C' for face IDs and names
  ws.column_dimensions['B'].width = 15  # Adjust this as necessary
  ws.column_dimensions['C'].width = 20  # Adjust this as necessary
  for row in range(faces.shape[0]):
    ws.row_dimensions[row+2].height = 60



  # Save the workbook as an Excel file
  wb.save('images_from_numpy.xlsx')

  print("Excel file with images saved!")
  return


In [ ]:
excel_faces(dir_=output_dir,faces=faces)

In [ ]:
#create dictionary

dic_ = {'tagged': faces,
        'encodings': face_encoding_,

         }

In [ ]:
# Load the image where you want to find and tag the face
#image_to_tag_path = 'Salo_predict.JPG'  # Replace with the path to the image where you want to find the face
#image_to_tag = face_recognition.load_image_file(image_to_tag_path)



# Detect faces in the new image
#face_locations = face_recognition.face_locations(image_to_tag)
#face_encodings = face_recognition.face_encodings(image_to_tag, face_locations)
#linew_ = 5
# Convert image to BGR format for OpenCV display (face_recognition uses RGB by default)
#image_to_tag_bgr = cv2.cvtColor(image_to_tag, cv2.COLOR_RGB2BGR)

# Loop over each face found in the image
#i = 0
#for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
#    print('face :',i+1)
#    i+=1
    # Check if the detected face matches the labeled face
#    matches = face_recognition.compare_faces([labeled_face_encoding], face_encoding)

    # If a match is found, tag the face
 #   if matches[0].any():
        # Draw a rectangle around the matching face
  #      cv2.rectangle(image_to_tag, (left, top), (right, bottom), (0, 255, 0), linew_)
   #     cv2.putText(image_to_tag, labeled_image_path[:-4], (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 255, 0), linew_)



In [ ]:
#plt.imshow(labeled_image)
#plt.show()

# Display the result
#plt.imshow(image_to_tag)
plt.show()


# Optionally, save the output image
#cv2.imwrite('tagged_image.jpg', image_to_tag_bgr)


In [ ]:
#face_locations